In [71]:
import pycaret
import pandas as pd

In [72]:
fake_news = pd.read_csv("Fake.csv")
real_news = pd.read_csv("True.csv")
fake_news['fake'] = 1
real_news['fake'] = 0
news = pd.concat([fake_news, real_news])

In [73]:
news.head()

,title,text,subject,date,fake
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [74]:
news.shape

(44898, 5)

In [75]:
news['all_text'] = news['title'] + news['text']
news.drop_duplicates(subset=['all_text'], inplace = True)
news.shape

(39105, 6)

In [76]:
news.head()

,title,text,subject,date,fake,all_text
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1,Donald Trump Sends Out Embarrassing New Year’...
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1,Drunk Bragging Trump Staffer Started Russian ...
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1,Sheriff David Clarke Becomes An Internet Joke...
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1,Trump Is So Obsessed He Even Has Obama’s Name...
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1,Pope Francis Just Called Out Donald Trump Dur...


In [77]:
# news = news.sample(1000, random_state=786).reset_index(drop=True)
# news.shape

In [78]:
modified_news = news.drop(['title','text','subject','date'], axis=1).reset_index(drop=True)
modified_news.head()

,fake,all_text
0,1,Donald Trump Sends Out Embarrassing New Year’...
1,1,Drunk Bragging Trump Staffer Started Russian ...
2,1,Sheriff David Clarke Becomes An Internet Joke...
3,1,Trump Is So Obsessed He Even Has Obama’s Name...
4,1,Pope Francis Just Called Out Donald Trump Dur...


In [79]:
from pycaret.nlp import *

In [80]:
fake_news_nlp = setup(data = modified_news, target = 'all_text', session_id = 123)

Description,Value
session_id,123
Documents,39105
Vocab Size,75475
Custom Stopwords,False


In [81]:
lda = create_model('lda',multi_core=True)

In [82]:
lda_df = assign_model(lda)
lda_df.head()

,fake,all_text,Topic_0,Topic_1,Topic_2,Topic_3,Dominant_Topic,Perc_Dominant_Topic
0,1,send embarrass new year message wish happy new...,0.926613,0.014157,0.057599,0.001631,Topic 0,0.93
1,1,drunk bragging trump staffer start russian col...,0.375756,0.002352,0.619573,0.002319,Topic 2,0.62
2,1,become internet joke threaten poke people eye ...,0.701940,0.001383,0.240402,0.056276,Topic 0,0.70
3,1,obsess even name code website image announce b...,0.726390,0.001629,0.270359,0.001622,Topic 0,0.73
4,1,call speechpope use annual message even mentio...,0.568869,0.208885,0.001700,0.220546,Topic 0,0.57


In [83]:
plot_model(lda, plot = 'topic_distribution')

In [84]:
evaluate_model(lda)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Frequency Plot', 'freque…

In [85]:
lda_df.drop(['all_text','Dominant_Topic','Perc_Dominant_Topic'], axis=1, inplace=True)
lda_df.head()

,fake,Topic_0,Topic_1,Topic_2,Topic_3
0,1,0.926613,0.014157,0.057599,0.001631
1,1,0.375756,0.002352,0.619573,0.002319
2,1,0.701940,0.001383,0.240402,0.056276
3,1,0.726390,0.001629,0.270359,0.001622
4,1,0.568869,0.208885,0.001700,0.220546


# Classification

In [86]:
from pycaret.classification import *

In [87]:
pce_1 = setup(data = lda_df, target = 'fake', session_id=5, train_size=0.85, silent=True)

,Description,Value
0,session_id,5
1,Target,fake
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(39105, 5)"
5,Missing Values,False
6,Numeric Features,4
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [97]:
top1 = compare_models(n_select=1)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.8845,0.9520,0.8847,0.8659,0.8752,0.7678,0.7680,0.0760
gbc,Gradient Boosting Classifier,0.8833,0.9523,0.8858,0.8629,0.8742,0.7655,0.7657,1.3040
ada,Ada Boost Classifier,0.8799,0.9496,0.8885,0.8548,0.8713,0.7588,0.7593,0.3140
rf,Random Forest Classifier,0.8768,0.9440,0.8700,0.8622,0.8660,0.7521,0.7522,1.3040
lr,Logistic Regression,0.8751,0.9455,0.8411,0.8806,0.8604,0.7475,0.7482,0.0240
svm,SVM - Linear Kernel,0.8743,0.0000,0.8516,0.8712,0.8611,0.7464,0.7469,0.0140
lda,Linear Discriminant Analysis,0.8692,0.9457,0.8155,0.8894,0.8508,0.7347,0.7369,0.0150
ridge,Ridge Classifier,0.8691,0.0000,0.8154,0.8894,0.8508,0.7346,0.7369,0.0110
nb,Naive Bayes,0.8676,0.9407,0.8176,0.8843,0.8496,0.7316,0.7335,0.0120
knn,K Neighbors Classifier,0.8651,0.9233,0.8596,0.8478,0.8536,0.7285,0.7286,0.0540


In [98]:
pce_1_m1 = create_model(top1)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8824,0.9525,0.8883,0.8595,0.8737,0.7637,0.7640
1,0.8956,0.9563,0.8928,0.8807,0.8867,0.7899,0.7900
2,0.8872,0.9515,0.8889,0.8678,0.8782,0.7732,0.7734
3,0.8866,0.9535,0.8909,0.8653,0.8779,0.7720,0.7723
4,0.8821,0.9480,0.8777,0.8663,0.8720,0.7627,0.7627
5,0.8745,0.9452,0.8738,0.8552,0.8644,0.7477,0.7479
6,0.8866,0.9558,0.8836,0.8705,0.8770,0.7718,0.7719
7,0.8818,0.9495,0.8797,0.8643,0.8719,0.7622,0.7623
8,0.8881,0.9559,0.8922,0.8671,0.8795,0.7751,0.7753


In [90]:
# blender = blend_models(top3)

# # blend model lousier than lr model

In [99]:
tuned_pce_1_m1 = tune_model(pce_1_m1, n_iter=50, early_stopping = True)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8827,0.9526,0.8883,0.8601,0.8739,0.7643,0.7646
1,0.8941,0.9566,0.8961,0.8754,0.8856,0.7871,0.7873
2,0.8878,0.9524,0.8948,0.8647,0.8795,0.7746,0.7750
3,0.8860,0.9530,0.8935,0.8623,0.8776,0.7710,0.7714
4,0.8806,0.9477,0.8744,0.8659,0.8701,0.7596,0.7596
5,0.8773,0.9451,0.8817,0.8547,0.8680,0.7533,0.7537
6,0.8866,0.9558,0.8895,0.8662,0.8777,0.7720,0.7722
7,0.8800,0.9503,0.8790,0.8615,0.8702,0.7586,0.7587
8,0.8884,0.9562,0.8922,0.8676,0.8797,0.7757,0.7759


In [103]:
cali_model = calibrate_model(tuned_pce_1_m1)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8827,0.9527,0.8778,0.8675,0.8726,0.7639,0.7639
1,0.8920,0.9566,0.8869,0.8783,0.8826,0.7826,0.7826
2,0.8899,0.9535,0.8935,0.8695,0.8813,0.7787,0.7789
3,0.8845,0.9533,0.8863,0.8647,0.8753,0.7677,0.7679
4,0.8794,0.9480,0.8685,0.8679,0.8682,0.7570,0.7570
5,0.8758,0.9455,0.8731,0.8579,0.8654,0.7500,0.7501
6,0.8854,0.9561,0.8777,0.8725,0.8751,0.7692,0.7692
7,0.8782,0.9506,0.8718,0.8633,0.8675,0.7547,0.7548
8,0.8857,0.9567,0.8856,0.8674,0.8764,0.7701,0.7702


In [ ]:
final_model = finalize_model(cali_model)

In [105]:
evaluate_model(final_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…